### Requierement: pip install lxml

In [1]:
from lxml import etree as ET
import csv
import os
import copy


### Modified code to handle both versions of DTD: DTD v1.3 and v1.0

In [2]:

# Change working directory
#os.chdir("/Users/tillohlendorf/Documents/MBT/Module/Systems_BioMedicine/NLP/TFA_repo/sysbiomed_nlp_project/Till")

def extract_xml_files(input_list_pmids, input_list_pmcids, input_dir, output_file):
    CRC_root = ET.Element("CRC_articles")
    previously_found_pmids = []
    previously_found_pmcids = []

    for subdir, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.xml'):
                xml_file_path = os.path.join(subdir, file)
                try:
                    parser = ET.XMLParser(recover=True)  # 'recover' allows for parsing despite errors
                    tree = ET.parse(xml_file_path, parser)
                    root = tree.getroot()
                    root_copy = copy.deepcopy(root)

                    for element in root_copy.xpath('.//article-id'):
                        pub_id_type = element.get('pub-id-type')
                        text = element.text

                        if ((pub_id_type == 'pmid' and text in input_list_pmids and text not in previously_found_pmids) or 
                            (pub_id_type == 'pmc' and text in input_list_pmcids and text not in previously_found_pmcids)):

                            CRC_root.append(root_copy)
                            # Keep track of added pmids and pmcids
                            if pub_id_type == 'pmid':
                                previously_found_pmids.append(text)
                            if pub_id_type == 'pmc':
                                previously_found_pmcids.append(text)
                            
                            break  # Break the loop once a valid 'article-id' is found
                except ET.ParseError as e:
                    print(f"Skipping file {xml_file_path} due to parse error: {e}")

    CRC_tree = ET.ElementTree(CRC_root)
    CRC_tree.write(output_file, encoding='utf-8', xml_declaration=True)

# Example usage
# Define your input lists, input directory, and output file
# extract_xml_files(input_list_pmids, input_list_pmcids, input_dir, output_file)


In [3]:
test_pmids = ["34035045", "34035044", "33888511", "12929205"] # first three IDs version 1.3 and last one 1.0
test_pmcids = ["PMC8494224", "PMC8494231", "PMC8494234", "PMC176545"] # first three IDs version 1.3 and last one 1.0

In [5]:
input_dir = '/Users/zeynepkorkmaz/Downloads/All_Articles/PMC000xxxxxx'

output_file = "/Users/zeynepkorkmaz/Downloads/Error_XML_test.xml"

In [7]:
extract_xml_files(test_pmids, test_pmcids, input_dir, output_file )